In [1]:
import multiprocessing as mp
import numpy as np
import pod5 as p5
import pysam
from multiprocessing import Queue

In [20]:
def extract_signal_from_pod5(pod5_path):
    signals={}
    with p5.Reader(pod5_path) as reader:
        for read_record in reader.reads():
            signals[str(read_record.read_id)] = {'signal':read_record.signal,'shift':read_record.calibration.offset,'scale':read_record.calibration.scale}#不加str会变成UUID，很奇怪
    return signals

def extract_move_from_bam(bam_path):
    read_dict=dict()
    bamfile = pysam.AlignmentFile(bam_path, "rb",check_sq=False)
    try:
        for read in bamfile.fetch():
            
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            read_dict.update({read.query_name:{"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}})
    except ValueError:
        print('bam don\'t has index')
        for read in bamfile.fetch(until_eof=True,multiple_iterators=True):
            tags=dict(read.tags)
            mv_tag=tags['mv']
            ts_tag=tags['ts']
            sm_tag=tags["sm"]
            sd_tag=tags["sd"]
            read_dict[read.query_name] = {"sequence":read.query_sequence,"stride":mv_tag[0],"mv_table":np.array(mv_tag[1:]),"num_trimmed":ts_tag,"shift":sm_tag,"scale":sd_tag}
    return read_dict

In [25]:
bam_path = '/homeb/xiaoyf/data/HG002/example/bam/sorted_has_moves.bam'
bamfile = pysam.AlignmentFile(bam_path, "rb",check_sq=False)
bamfile.has_index()

True

In [9]:
for read in bamfile.get_index_statistics():
    #print(read)
    print(type(read))

In [22]:
seq_and_move_table=extract_move_from_bam(bam_path)

In [24]:
seq_and_move_table

{}

In [26]:
base2code_dna = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4,
                 'W': 4, 'S': 4, 'M': 4, 'K': 4, 'R': 4,
                 'Y': 4, 'B': 4, 'V': 4, 'D': 4, 'H': 4,
                 'Z': 4}  # set 4 for all bases except ACGT, for now
def norm_signal_read_id(signal):
    shift_scale_norm={}
    signal_norm={}
    shift_scale_norm={}
    shift_scale_norm['shift']=(signal['to_norm_shift']/signal['to_pA_scale'])-signal['to_pA_shift']
    shift_scale_norm['scale']=(signal['to_norm_scale']/signal['to_pA_scale'])
    num_trimmed=signal["num_trimmed"]
    signal_norm=(signal['signal'][num_trimmed:] - shift_scale_norm['shift']) / shift_scale_norm['scale']        
    return signal_norm

def read_from_pod5_bam(pod5_path,bam_path,read_id=None):
    read={}
    signal = extract_signal_from_pod5(pod5_path)
    seq_move = extract_move_from_bam(bam_path)
    if read_id is not None:
        if seq_move[read_id]['sequence'] is not None:
            if signal[read_id] is not None:
                read[read_id]={"sequence":seq_move[read_id]['sequence'],"signal":signal[read_id]['signal'],'mv_table':seq_move[read_id]['mv_table'],
                               "num_trimmed":seq_move[read_id]['num_trimmed'],"to_norm_shift":seq_move[read_id]['shift'],
                               "to_norm_scale":seq_move[read_id]['scale'],"stride":seq_move[read_id]['stride'],
                               'to_pA_shift':signal[read_id]['shift'],
                               'to_pA_scale':signal[read_id]["scale"]}
    else:
        for read_id in seq_move.keys():
            if seq_move[read_id]['sequence'] is not None:
                if signal[read_id] is not None:
                    read[read_id]={"sequence":seq_move[read_id]['sequence'],"signal":signal[read_id]['signal'],'mv_table':seq_move[read_id]['mv_table'],
                               "num_trimmed":seq_move[read_id]['num_trimmed'],"to_norm_shift":seq_move[read_id]['shift'],
                               "to_norm_scale":seq_move[read_id]['scale'],"stride":seq_move[read_id]['stride'],
                               'to_pA_shift':signal[read_id]['shift'],
                               'to_pA_scale':signal[read_id]["scale"]}
    return read

def caculate_feature_for_each_base(read,base_num=0):   
    feature={}
    for read_id in read.keys():
        feature[read_id]=[]
        sequence = read[read_id]['sequence']
        movetable = read[read_id]['mv_table']
        stride = read[read_id]['stride']
        #num_trimmed = read[read_id]['num_trimmed']
        trimed_signals = norm_signal_read_id(read[read_id])#筛掉背景信号,norm
        move_pos = np.append(np.argwhere(movetable == 1).flatten(), len(movetable))
        #print(len(move_pos))
        for move_idx in range(len(move_pos) - 1):
            start, end = move_pos[move_idx], move_pos[move_idx + 1]
            signal=trimed_signals[(start * stride):(end * stride)].tolist()
            mean=np.mean(signal)
            std=np.std(signal)
            num=end-start        
            feature[read_id].append({'signal':signal,'std':std,'mean':mean,'num':num*read[read_id][stride],'base':sequence[move_idx]})
    if base_num==0:
        return feature
    else:
        windows_size=base_num/2
        for read_id in feature.keys():
            for i in range(len(feature[read_id])):
                nbase=[]
                nsig=[]
                if i<windows_size:                   
                    if i!=0:
                        for k in range(i):
                            nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                            nsig=nsig+feature[read_id][k]['signal']
                    nbase=nbase+feature[read_id][i]['base']*(windows_size-i)*feature[read_id][k][num]
                    nbase=nbase+feature[read_id][i]['base']*feature[read_id][k][num]
                    for k in range(i,i+windows_size):
                        nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                    nsig=nsig+feature[read_id][i]['signal']*(windows_size-i)
                    nsig=nsig+feature[read_id][i]['signal']
                    for k in range(i,i+windows_size):
                        nsig=nsig+feature[read_id][k]['signal']
                elif (len(feature[read_id])-1)-i<windows_size:
                    for k in range(i-windows_size,i):
                        nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                    nbase=nbase+feature[read_id][i]['base']*feature[read_id][k][num]
                    for k in range(i-windows_size,i):
                        nsig=nsig+feature[read_id][k]['signal']
                    nsig=nsig+feature[read_id][i]['signal']                   
                    if i!=len(feature[read_id])-1:
                        for k in range(i,len(feature[read_id])-1):
                            nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                            nsig=nsig+feature[read_id][k]['signal']
                    nbase=nbase+feature[read_id][i]['base']*(windows_size-((len(feature[read_id])-1)-i))*feature[read_id][k][num]
                    nsig=nsig+feature[read_id][i]['signal']*(windows_size-((len(feature[read_id])-1)-i))
                else:
                    for k in range(i-windows_size,i):
                        nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                    nbase=nbase+feature[read_id][i]['base']*feature[read_id][k][num]
                    for k in range(i,i+windows_size):
                        nbase=nbase+feature[read_id][k]['base']*feature[read_id][k][num]
                    for k in range(i-windows_size,i):
                        nsig=nsig+feature[read_id][k]['signal']
                    nsig=nsig+feature[read_id][i]['signal']
                    for k in range(i,i+windows_size):
                        nsig=nsig+feature[read_id][k]['signal']
                feature[read_id][i].update({'nbase':nbase,'nsig':nsig})
        return feature


In [37]:
pod5_path = '/homeb/xiaoyf/data/HG002/example/pod5/output.pod5'
bam_path = '/homeb/xiaoyf/data/HG002/example/bam/has_moves.bam'
read=read_from_pod5_bam(pod5_path,bam_path)

[E::idx_find_and_load] Could not retrieve index file for '/homeb/xiaoyf/data/HG002/example/bam/has_moves.bam'


bam don't has index


In [38]:
feature=caculate_feature_for_each_base(read,21)

In [43]:
feature['0000b1ad-fdaf-49e6-bc11-cbe93270e3a3'][0].keys()

dict_keys(['signal', 'std', 'mean', 'num'])

In [40]:
def write_feature(feature):
    with open('/homeb/xiaoyf/data/HG002/example/feature.txt','w') as f:
        for read_id in feature.keys():
            #f.write(read_id+'\t')
            for i in range(len(feature[read_id])):
                f.write(str(feature[read_id][i]['nbase'])+'\t'+str(feature[read_id][i]['nsig'])+'\n')

In [41]:
write_feature(feature)

KeyError: 'nbase'